In [ ]:
from pathlib import Path
import random
import shutil
import time

from cytoolz import first
from gPhoton.coadd import coadd_galex_rice_slices
from gPhoton.reference import eclipse_to_paths
from killscreen.monitors import Netstat, Stopwatch
import pandas as pd
import pyarrow as pa
from pyarrow import pa_csv
from pyarrow import parquet
import sh

from s3_fuse.ps1_utils import prune_ps1_catalog, get_ps1_cutouts

In [ ]:
# 'configuration'

BUCKET_NAME = 'nishapur'
S3_ROOT = '/mnt/s3'

# desired cutout side length in degrees
CUTOUT_SIDE_LENGTH = 50 / 3600

# how many (randomly-selected) targets would we like cutouts for? 
TARGET_COUNT = 10
# which PS1 bands are we looking at? (currently only g and z are staged.)
PS1_BANDS = ("g", "z")
# which GALEX bands are we looking at? (currently only NUV is staged.)
GALEX_BANDS = ("NUV",)

try:
    sh.umount(S3_ROOT)
except sh.ErrorReturnCode:
    pass
finally:
    sh.goofys(BUCKET_NAME, S3_ROOT)

In [ ]:
catalog_fn = "ps1_extragalactic_skycells_eclipses.parquet"
if not Path(catalog_fn).exists():
    shutil.copy(
        Path(S3_ROOT, "ps1/metadata", catalog_fn),
        Path(catalog_fn)
    )
catalog = parquet.read_table(catalog_fn)

In [ ]:
test_cell_fn = "ps1_extragalactic_skycells_eclipses_1k_cell_subset.csv"
arbitrary_test_cells = (
    pa_csv
    .read_csv(Path(S3_ROOT, "ps1/metadata", test_cell_fn))
    .cast(pa.schema([("proj_cell", pa.uint16()), ("sky_cell", pa.uint8())]))
)
# table of actually-existing mislike eclipse images, excluding those with 
# all-flagged data, bad distortion correction solutions, etc.
extant_mislike = pd.read_csv(Path(S3_ROOT, "extant_mislike_eclipses.csv"))['0']
small_catalog = prune_ps1_catalog(catalog, arbitrary_test_cells)


In [ ]:
TARGET_COUNT = 20
targets = small_catalog.take(
    random.sample(range(len(small_catalog)), k=TARGET_COUNT)
).to_pylist()
stat, watch = Netstat(), Stopwatch()
for target in targets:
    print(
        f"getting square cutouts with {round(CUTOUT_SIDE_LENGTH, 2)}-"
        f"degree sides around PS1 object {target['obj_id']} "
        f"with mean ra {round(target['ra'], 3)} and mean "
        f"dec {round(target['dec'], 3)}\n"
    )
    galex_paths = [
        # note that these test files do not have 'rice' in the name
        eclipse_to_paths(eclipse, S3_ROOT, None, "none")
        for eclipse in target['galex']
        if eclipse in extant_mislike.values
    ]
    if len(galex_paths) == 0:
        print("...all specified GALEX eclipses for this target are bad, skipping...")
    else:
        coadd, wcs = coadd_galex_rice_slices(
            image_paths = [paths['NUV']['image'] for paths in galex_paths],
            ra = target['ra'],
            dec = target['dec'],
            side_length = CUTOUT_SIDE_LENGTH,
            stat=stat,
            watch=watch
        )
    print("\n")
    ps1_cutouts, ps1_wcs = get_ps1_cutouts(
        target, PS1_BANDS, CUTOUT_SIDE_LENGTH, S3_ROOT, stat, watch
    )
    print("\n")
print(f"{round(time.time() - watch.start_time, 2)} total seconds")
print(
    f"{round(first(stat.total.values()) / 1024 ** 2)} MB transferred"
)
